In [1]:
import pandas as pd
import numpy as np
import re

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

--------------

### Combining the data files into one data frame

In [2]:
file_name1 = "games_data/lowest_rated_games.csv"
file_name2 = "games_data/top_rated_games.csv"
df = pd.concat(
    map(pd.read_csv, [file_name1, file_name2]), ignore_index=True)

## <font color='red'>Problem1:</font> Incomplete data
See problem: [4_EDA](./4_EDA.ipynb)

Following this problem, which was addressed in the previous step by scraping all the data

We created a new data frame and saved it in 'all_games.csv' file

See previous step: [game_crawling](./2_game_crawling.ipynb) => <font color='red'>'Problem1: Incomplete data'</font>

And then we will save everything in a new file: [handled_games.csv](#Incomplete_data)

In [3]:
df = pd.read_csv("games_data/all_games.csv")

# Deletion of the index column - "Unnamed: 0"
df.drop("Unnamed: 0", axis=1, inplace=True)

# Now we will handle the data in the new file

<a id='Improving_Performance'></a>
## <font color='red'>Problem2:</font> Improving Performance
See problem: [machine_learning](./5_machine_learning.ipynb)

## At the machine learning stage we saw that the ability to predict was not the best
### f1 = 0.456

### So we came back here and made some changes:

- [Dropping the nan values for 'user_score' and Not filling the nan values with the mean values](#Improving_Performance1)
- [Handling Duplicates in 'genres'](#Improving_Performance2)

Finally we saved everything in a new file: [all_games3.csv](#Improving_Performance4)

--------------
Checking the data

In [4]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,Nintendo,"Nov 23, 1998",99.0,9.1,Nintendo,"Action Adventure, Fantasy",1 Player,E,"6,736",529,270,22.0,0.0,0.0
1,Tony Hawk's Pro Skater 2,PlayStation,Activision,"Sep 20, 2000",98.0,7.5,Neversoft Entertainment,"Sports, Alternative, Skateboarding",1-2,T,609,150,112,19.0,0.0,0.0
2,Grand Theft Auto IV,PlayStation 3,Rockstar Games,"Apr 29, 2008",98.0,7.8,Rockstar North,"Action Adventure, Modern, Modern, Open-World",1 Player,M,"3,450",607,770,64.0,0.0,0.0
3,SoulCalibur,Dreamcast,Namco,"Sep 8, 1999",98.0,8.4,Namco,"Action, Fighting, 3D",1-2,T,335,40,31,24.0,0.0,0.0
4,Grand Theft Auto IV,Xbox 360,Rockstar Games,"Apr 29, 2008",98.0,8.0,Rockstar North,"Action Adventure, Modern, Modern, Open-World",1 Player,M,"2,687",451,555,86.0,0.0,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19223 entries, 0 to 19222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               19223 non-null  object 
 1   platform                19223 non-null  object 
 2   publisher               19211 non-null  object 
 3   release_date            19223 non-null  object 
 4   meta_scroe              19214 non-null  float64
 5   user_score              19220 non-null  object 
 6   develeoper              19202 non-null  object 
 7   genres                  19223 non-null  object 
 8   num_of_players          15494 non-null  object 
 9   rating                  17111 non-null  object 
 10  user_positive_review    17950 non-null  object 
 11  user_negative_review    17950 non-null  object 
 12  user_mixed_review       17950 non-null  object 
 13  critic_positive_review  19216 non-null  float64
 14  critic_negative_review  19216 non-null

--------------

## In user_score there are some 'tbd' values 
'tbd' = to be declared

In [6]:
df[df['user_score'] == "tbd"].head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
1095,Atari 50: The Anniversary Celebration,Xbox Series X,Atari,"Nov 11, 2022",87.0,tbd,Digital Eclipse,"Miscellaneous, Compilation",No Online Multiplayer,T,NaN,NaN,NaN,9.0,0.0,0.0
1548,Pinball FX 2: Marvel Pinball - Vengeance and V...,Xbox 360,Zen Studios,"Dec 13, 2011",84.0,tbd,Zen Studios,"Pinball, Action, Miscellaneous, Parlor, Pinball",NaN,E,NaN,NaN,NaN,7.0,0.0,0.0
1612,NeverAwake,PC,Neotro Inc.,"Sep 28, 2022",84.0,tbd,Neotro Inc.,"Action, General, Shooter, Shoot-'Em-Up, Horizo...",No Online Multiplayer,NaN,NaN,NaN,NaN,9.0,0.0,0.0
1638,This Way Madness Lies,PC,Zeboyd Games,"Nov 10, 2022",84.0,tbd,Zeboyd Games,"Action, General, Role-Playing, Japanese-Style",No Online Multiplayer,NaN,NaN,NaN,NaN,6.0,0.0,1.0
1658,NASCAR Thunder 2003,Xbox,EA Sports,"Sep 19, 2002",84.0,tbd,EA Sports,"Driving, Racing, Stock Car",1-4,E,2,0,0,12.0,0.0,1.0


### Changing the 'tbd' to nan values:

In [7]:
df.replace("tbd", np.nan, inplace=True)
df[df["user_score"] == "tbd"]

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review


--------------
Checking the data:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19223 entries, 0 to 19222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               19223 non-null  object 
 1   platform                19223 non-null  object 
 2   publisher               19211 non-null  object 
 3   release_date            19223 non-null  object 
 4   meta_scroe              19214 non-null  float64
 5   user_score              17843 non-null  object 
 6   develeoper              19202 non-null  object 
 7   genres                  19223 non-null  object 
 8   num_of_players          15494 non-null  object 
 9   rating                  17111 non-null  object 
 10  user_positive_review    17950 non-null  object 
 11  user_negative_review    17950 non-null  object 
 12  user_mixed_review       17950 non-null  object 
 13  critic_positive_review  19216 non-null  float64
 14  critic_negative_review  19216 non-null

--------------

## Dropping the nan values for: 'meta_scroe', 'publisher', 'develeoper':

This information can not be determined and replaced

In [9]:
df.dropna(subset=['meta_scroe'], inplace=True)
df.dropna(subset=['publisher'], inplace=True)
df.dropna(subset=['develeoper'], inplace=True)

--------------
Checking the data:

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19183 entries, 0 to 19222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               19183 non-null  object 
 1   platform                19183 non-null  object 
 2   publisher               19183 non-null  object 
 3   release_date            19183 non-null  object 
 4   meta_scroe              19183 non-null  float64
 5   user_score              17814 non-null  object 
 6   develeoper              19183 non-null  object 
 7   genres                  19183 non-null  object 
 8   num_of_players          15470 non-null  object 
 9   rating                  17091 non-null  object 
 10  user_positive_review    17921 non-null  object 
 11  user_negative_review    17921 non-null  object 
 12  user_mixed_review       17921 non-null  object 
 13  critic_positive_review  19183 non-null  float64
 14  critic_negative_review  19183 non-null

--------------

In [12]:
df[df["user_score"].isnull()]

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
1095,Atari 50: The Anniversary Celebration,Xbox Series X,Atari,"Nov 11, 2022",87.0,NaN,Digital Eclipse,"Miscellaneous, Compilation",No Online Multiplayer,T,NaN,NaN,NaN,9.0,0.0,0.0
1548,Pinball FX 2: Marvel Pinball - Vengeance and V...,Xbox 360,Zen Studios,"Dec 13, 2011",84.0,NaN,Zen Studios,"Pinball, Action, Miscellaneous, Parlor, Pinball",NaN,E,NaN,NaN,NaN,7.0,0.0,0.0
1612,NeverAwake,PC,Neotro Inc.,"Sep 28, 2022",84.0,NaN,Neotro Inc.,"Action, General, Shooter, Shoot-'Em-Up, Horizo...",No Online Multiplayer,NaN,NaN,NaN,NaN,9.0,0.0,0.0
1638,This Way Madness Lies,PC,Zeboyd Games,"Nov 10, 2022",84.0,NaN,Zeboyd Games,"Action, General, Role-Playing, Japanese-Style",No Online Multiplayer,NaN,NaN,NaN,NaN,6.0,0.0,1.0
1658,NASCAR Thunder 2003,Xbox,EA Sports,"Sep 19, 2002",84.0,NaN,EA Sports,"Driving, Racing, Stock Car",1-4,E,2,0,0,12.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19040,Sanity of Morris,Xbox One,StickyLock Studios,"Mar 23, 2021",35.0,NaN,Alterego Games,"Action Adventure, Survival",No Online Multiplayer,NaN,0,0,2,0.0,5.0,2.0
19050,King of Clubs,Wii,Crave,"Aug 4, 2008",35.0,NaN,Oxygen Interactive,"Sports, Traditional, Individual, Golf, Arcade,...",No Online Multiplayer,E,NaN,NaN,NaN,1.0,9.0,1.0
19111,Jenga World Tour,DS,Atari,"Nov 13, 2007",32.0,NaN,Atomic Planet Entertainment,"Miscellaneous, Puzzle, Party, General",2 Online,E,2,1,0,0.0,6.0,1.0
19126,Dream Chronicles,PlayStation 3,Hudson Entertainment,"Nov 23, 2010",31.0,NaN,PlayFirst,"Miscellaneous, Puzzle, Puzzle, Hidden Object, ...",NaN,E,NaN,NaN,NaN,0.0,6.0,2.0


<a id='Improving_Performance1'></a>
## <font color='red'>Problem2:</font> Not filling the 'user_score' nan values with the mean values
See problem in page: [Problem2: Improving Performance](#Improving_Performance)

## Dropping the nan values for 'user_score'

This information can not be determined and replaced

In [13]:
df["user_score"] = df["user_score"].astype("float64")
df["user_score"].mean()

6.928544964634617

In [14]:
df.dropna(subset=['user_score'], inplace=True)
# df["user_score"].fillna(df["user_score"].mean(), inplace=True)

--------------
Checking the data:

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17814 entries, 0 to 19222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               17814 non-null  object 
 1   platform                17814 non-null  object 
 2   publisher               17814 non-null  object 
 3   release_date            17814 non-null  object 
 4   meta_scroe              17814 non-null  float64
 5   user_score              17814 non-null  float64
 6   develeoper              17814 non-null  object 
 7   genres                  17814 non-null  object 
 8   num_of_players          14382 non-null  object 
 9   rating                  15902 non-null  object 
 10  user_positive_review    17253 non-null  object 
 11  user_negative_review    17253 non-null  object 
 12  user_mixed_review       17253 non-null  object 
 13  critic_positive_review  17814 non-null  float64
 14  critic_negative_review  17814 non-null

In [16]:
df.describe(include="all")

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_review,user_negative_review,user_mixed_review,critic_positive_review,critic_negative_review,critic_mixed_review
count,17814,17814,17814,17814,17814.000000,17814.000000,17814,17814,14382,15902,17253,17253,17253,17814.000000,17814.000000,17814.000000
unique,11695,22,1911,4580,NaN,NaN,3702,2184,56,7,1168,594,468,NaN,NaN,NaN
top,Madden NFL 07,PC,Ubisoft,"Nov 14, 2006",NaN,NaN,Capcom,"Action, Platformer, 2D",No Online Multiplayer,T,4,2,2,NaN,NaN,NaN
freq,9,4816,777,44,NaN,NaN,335,490,6208,5452,731,1827,1634,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,70.639329,6.928545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.068036,1.570001,8.648872
std,NaN,NaN,NaN,NaN,12.027933,1.359784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.641455,3.376621,7.781927
min,NaN,NaN,NaN,NaN,11.000000,0.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,64.000000,6.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,3.000000
50%,NaN,NaN,NaN,NaN,73.000000,7.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,0.000000,6.000000
75%,NaN,NaN,NaN,NaN,79.000000,7.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.000000,2.000000,12.000000


--------------

### Data with large numbers are with commas

In [17]:
df["user_positive_review"][20]

'6,414'

Therefore, a simple conversion to the int type will not help here

# Review Data handling

### Conversion of all review data to int type without the commas

In [18]:
colums = ["user_positive_review", "user_negative_review", "user_mixed_review"]
for column in colums:
    nums = []
    for string in df[column]:
        if(type(string) != float):
            nums.append(int(string.replace(",", "")))
        else:
            nums.append(np.nan)
    df[column] = nums

--------------
Checking the data:

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17814 entries, 0 to 19222
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_name               17814 non-null  object 
 1   platform                17814 non-null  object 
 2   publisher               17814 non-null  object 
 3   release_date            17814 non-null  object 
 4   meta_scroe              17814 non-null  float64
 5   user_score              17814 non-null  float64
 6   develeoper              17814 non-null  object 
 7   genres                  17814 non-null  object 
 8   num_of_players          14382 non-null  object 
 9   rating                  15902 non-null  object 
 10  user_positive_review    17253 non-null  float64
 11  user_negative_review    17253 non-null  float64
 12  user_mixed_review       17253 non-null  float64
 13  critic_positive_review  17814 non-null  float64
 14  critic_negative_review  17814 non-null

--------------

## Filling the review data nan values with the mean values

In [20]:
columns = [
    "user_positive_review", "user_negative_review", "user_mixed_review",
    "critic_positive_review", "critic_negative_review", "critic_mixed_review"
]
for column in columns:
    df[column] = df[column].astype("float")
    df[column].fillna(df[column].mean(), inplace=True)

## Normalizing the review data

In [21]:
# making a new normalize column of 'user_positive_review'
user_review = df.iloc[:,10:13].copy()
user_review_sum = user_review["user_positive_review"] + user_review["user_negative_review"] + user_review["user_mixed_review"]
user_review["user_positive_normalize"] = user_review["user_positive_review"] / user_review_sum

df["user_positive_normelize"] = user_review["user_positive_normalize"]


In [22]:
# making a new normalize column of 'critic_positive_review'
critic_review = df.iloc[:,-4:-1].copy()
critic_review_sum = critic_review["critic_positive_review"] + critic_review["critic_mixed_review"] + critic_review["critic_negative_review"]
critic_review["critic_positive_normalize"] = critic_review["critic_positive_review"] / critic_review_sum

df["critic_positive_normelize"] = critic_review["critic_positive_normalize"]


In [23]:
df.drop(columns, axis=1, inplace=True)

--------------
Checking the data:

In [24]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,num_of_players,rating,user_positive_normelize,critic_positive_normelize
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,Nintendo,"Nov 23, 1998",99.0,9.1,Nintendo,"Action Adventure, Fantasy",1 Player,E,0.893962,1.0
1,Tony Hawk's Pro Skater 2,PlayStation,Activision,"Sep 20, 2000",98.0,7.5,Neversoft Entertainment,"Sports, Alternative, Skateboarding",1-2,T,0.699196,1.0
2,Grand Theft Auto IV,PlayStation 3,Rockstar Games,"Apr 29, 2008",98.0,7.8,Rockstar North,"Action Adventure, Modern, Modern, Open-World",1 Player,M,0.714730,1.0
3,SoulCalibur,Dreamcast,Namco,"Sep 8, 1999",98.0,8.4,Namco,"Action, Fighting, 3D",1-2,T,0.825123,1.0
4,Grand Theft Auto IV,Xbox 360,Rockstar Games,"Apr 29, 2008",98.0,8.0,Rockstar North,"Action Adventure, Modern, Modern, Open-World",1 Player,M,0.727593,1.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17814 entries, 0 to 19222
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  17814 non-null  object 
 1   platform                   17814 non-null  object 
 2   publisher                  17814 non-null  object 
 3   release_date               17814 non-null  object 
 4   meta_scroe                 17814 non-null  float64
 5   user_score                 17814 non-null  float64
 6   develeoper                 17814 non-null  object 
 7   genres                     17814 non-null  object 
 8   num_of_players             14382 non-null  object 
 9   rating                     15902 non-null  object 
 10  user_positive_normelize    17814 non-null  float64
 11  critic_positive_normelize  17814 non-null  float64
dtypes: float64(4), object(8)
memory usage: 2.3+ MB


--------------

# Adding 'max_players' and 'online_game' columns
With the information in the 'num_of_players' column

In [26]:
num_of_players_unique = list(df["num_of_players"].unique())

replace_map_for_max_players = {}
replace_map_for_online_game = {}

for val in num_of_players_unique:
    if(type(val) != float):
        max_player = np.nan
        max_player_list = [int(s) for s in re.findall(r'\b\d+\b', val)]
        if(max_player_list):
            max_player = max(max_player_list)
            replace_map_for_max_players[val] = max_player
        else:
            replace_map_for_max_players[val] = 0

        
        online = ('Online' in val) and ('No Online' not in val)
        if(max_player != np.nan):
            if(max_player >= 4):
                online = True
        
        if(online):
            replace_map_for_online_game[val] = 1
        else:
            replace_map_for_online_game[val] = 0
            
    else:
        replace_map_for_max_players[val] = 0
        replace_map_for_online_game[val] = 0
            
df["max_players"] = df["num_of_players"].copy()
df["online_game"] = df["num_of_players"].copy()

df["max_players"].replace(replace_map_for_max_players, inplace=True)
df["online_game"].replace(replace_map_for_online_game, inplace=True)

df.drop("num_of_players", axis=1, inplace=True)

### 'online_game' column:

1 = online

0 = not online

In [27]:
df["online_game"].unique()

array([0, 1], dtype=int64)

### 'max_players' column:

Maximum players possible in a game

In [28]:
df["max_players"].unique()

array([ 1,  2,  0, 32, 30, 16,  4,  6,  8, 18, 10, 40, 12,  5, 64, 20, 24,
       22, 60,  3,  9, 44, 14, 15, 36], dtype=int64)

--------------
Checking the data:

In [29]:
df.head()

,game_name,platform,publisher,release_date,meta_scroe,user_score,develeoper,genres,rating,user_positive_normelize,critic_positive_normelize,max_players,online_game
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,Nintendo,"Nov 23, 1998",99.0,9.1,Nintendo,"Action Adventure, Fantasy",E,0.893962,1.0,1,0
1,Tony Hawk's Pro Skater 2,PlayStation,Activision,"Sep 20, 2000",98.0,7.5,Neversoft Entertainment,"Sports, Alternative, Skateboarding",T,0.699196,1.0,2,0
2,Grand Theft Auto IV,PlayStation 3,Rockstar Games,"Apr 29, 2008",98.0,7.8,Rockstar North,"Action Adventure, Modern, Modern, Open-World",M,0.714730,1.0,1,0
3,SoulCalibur,Dreamcast,Namco,"Sep 8, 1999",98.0,8.4,Namco,"Action, Fighting, 3D",T,0.825123,1.0,2,0
4,Grand Theft Auto IV,Xbox 360,Rockstar Games,"Apr 29, 2008",98.0,8.0,Rockstar North,"Action Adventure, Modern, Modern, Open-World",M,0.727593,1.0,1,0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17814 entries, 0 to 19222
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  17814 non-null  object 
 1   platform                   17814 non-null  object 
 2   publisher                  17814 non-null  object 
 3   release_date               17814 non-null  object 
 4   meta_scroe                 17814 non-null  float64
 5   user_score                 17814 non-null  float64
 6   develeoper                 17814 non-null  object 
 7   genres                     17814 non-null  object 
 8   rating                     15902 non-null  object 
 9   user_positive_normelize    17814 non-null  float64
 10  critic_positive_normelize  17814 non-null  float64
 11  max_players                17814 non-null  int64  
 12  online_game                17814 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usag

--------------

## We will change the names of the platforms to make them more general

In [31]:
df["platform"].unique()

array(['Nintendo 64', 'PlayStation', 'PlayStation 3', 'Dreamcast',
       'Xbox 360', 'Wii', 'Xbox One', 'PC', 'Switch', 'PlayStation 2',
       'PlayStation 4', 'GameCube', 'Xbox', 'Wii U', 'Xbox Series X',
       'PlayStation 5', 'Game Boy Advance', '3DS', 'DS',
       'PlayStation Vita', 'PSP', 'Stadia'], dtype=object)

In [32]:
platform_replace_map = {
    'PSP': 'PlayStation',
    '3DS': 'Nintendo',
    'Switch': 'Nintendo',
    'DS': 'Nintendo',
    'Game Boy Advance': 'Nintendo',
    'Dreamcast': 'Old Platform',
    'GameCube': 'Old Platform',
    'Nintendo 64': 'Nintendo',
    'Stadia': 'PC'
}
platform_unique_list = list(df["platform"].unique())
for val in platform_unique_list:
    replaceXbox = re.findall(r'(Xbox) ', val)
    replacePlayStation = re.findall(r'(PlayStation) ', val)
    replaceWii = re.findall(r'(Wii) ', val)
    if(replaceXbox):
        platform_replace_map[val] = replaceXbox[0]
    if(replacePlayStation):
        platform_replace_map[val] = replacePlayStation[0]
    if(replaceWii):
        platform_replace_map[val] = replaceWii[0]

df["platform"].replace(platform_replace_map, inplace=True)

In [33]:
df["platform"].unique()

array(['Nintendo', 'PlayStation', 'Old Platform', 'Xbox', 'Wii', 'PC'],
      dtype=object)

## We will assign to each rating a value related to the order of the rating

In [34]:
df["rating"].unique()

array(['E', 'T', 'M', 'E10+', nan, 'K-A', 'AO', 'RP'], dtype=object)

First we replace the nan valuse with the top value:

In [35]:
top_val = df["rating"].describe()[2]
df["rating"].fillna(top_val, inplace=True)

The values will be distributed like this:
- RP - Rating Pending == 0
- E - Everyone 6 and older == 1
- E10+ - 10 and older == 2
- K-A - 10 and older == 2
- T - Teen 13 and older == 3
- m - 17+ == 4
- AO - Adults Only 18 years and older == 5

In [36]:
rating_replace_map = {
    'RP': 0, 'E': 1, 'E10+': 2,
    'K-A': 2, 'T': 3, 'M': 4, 'AO': 5}

df["rating"].replace(rating_replace_map, inplace=True)

## Scaling the 'meta_scroe' and 'user_score' columns

In [37]:
min_max_scaler = preprocessing.MinMaxScaler()
meta_scroe_scaled = min_max_scaler.fit_transform(df[['meta_scroe']])
user_score_scaled = min_max_scaler.fit_transform(df[['user_score']])
df['meta_score'] = meta_scroe_scaled
df['user_score'] = user_score_scaled
df.drop("meta_scroe", axis=1, inplace=True)

## The 'release_date' will be divided into two columns: 'release_month' and 'release_year'

In [38]:
df["release_date"].unique()

array(['Nov 23, 1998', 'Sep 20, 2000', 'Apr 29, 2008', ...,
       'Apr 25, 2001', 'Jul  6, 2006', 'Dec 21, 2011'], dtype=object)

In [39]:
import calendar

In [40]:
# Getting months by their numbers
month_number = {}
for index, month in enumerate(calendar.month_abbr):
    if(month):
        month_number[month.lower()] = index

# Divides into two columns:
release_date_list = list(df["release_date"])
release_year = []
release_month = []
for date in release_date_list:
    year = re.findall(r'\b\d+\b', date)[1]
    release_year.append(int(year))
    month = re.findall(r'(\w\w\w) ', date)[0]
    release_month.append(month_number[month.lower()])

df['release_year'] = release_year
df['release_month'] = release_month

In [41]:
df.drop("release_date", axis=1, inplace=True)

--------------
Checking the data:

In [42]:
df.head()

,game_name,platform,publisher,user_score,develeoper,genres,rating,user_positive_normelize,critic_positive_normelize,max_players,online_game,meta_score,release_year,release_month
0,The Legend of Zelda: Ocarina of Time,Nintendo,Nintendo,0.936170,Nintendo,"Action Adventure, Fantasy",1,0.893962,1.0,1,0,1.000000,1998,11
1,Tony Hawk's Pro Skater 2,PlayStation,Activision,0.765957,Neversoft Entertainment,"Sports, Alternative, Skateboarding",3,0.699196,1.0,2,0,0.988636,2000,9
2,Grand Theft Auto IV,PlayStation,Rockstar Games,0.797872,Rockstar North,"Action Adventure, Modern, Modern, Open-World",4,0.714730,1.0,1,0,0.988636,2008,4
3,SoulCalibur,Old Platform,Namco,0.861702,Namco,"Action, Fighting, 3D",3,0.825123,1.0,2,0,0.988636,1999,9
4,Grand Theft Auto IV,Xbox,Rockstar Games,0.819149,Rockstar North,"Action Adventure, Modern, Modern, Open-World",4,0.727593,1.0,1,0,0.988636,2008,4


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17814 entries, 0 to 19222
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  17814 non-null  object 
 1   platform                   17814 non-null  object 
 2   publisher                  17814 non-null  object 
 3   user_score                 17814 non-null  float64
 4   develeoper                 17814 non-null  object 
 5   genres                     17814 non-null  object 
 6   rating                     17814 non-null  int64  
 7   user_positive_normelize    17814 non-null  float64
 8   critic_positive_normelize  17814 non-null  float64
 9   max_players                17814 non-null  int64  
 10  online_game                17814 non-null  int64  
 11  meta_score                 17814 non-null  float64
 12  release_year               17814 non-null  int64  
 13  release_month              17814 non-null  int

--------------

# The genres are listed in long lists

In [44]:
df["genres"]

0                                Action Adventure, Fantasy
1                       Sports, Alternative, Skateboarding
2             Action Adventure, Modern, Modern, Open-World
3                                     Action, Fighting, 3D
4             Action Adventure, Modern, Modern, Open-World
                               ...                        
19218                           Sports, Individual, Biking
19219    Action Adventure, Adventure, Third-Person, Ope...
19220          Driving, Racing, Arcade, Arcade, Automobile
19221    Driving, Modern, Racing, Motorcycle, Motocross...
19222        Miscellaneous, Party, Party, Party / Minigame
Name: genres, Length: 17814, dtype: object

In [45]:
len(df["genres"].unique())

2184

### We will arrange the genres in order to minimize the appearance:

In [46]:
genres_per_game = []
for game_genres in df["genres"]:
    genre_list = sorted(game_genres.split(", "))
    genres_new_string = ", ".join(genre_list)
    genres_per_game.append(genres_new_string)

df["genres"] = genres_per_game

In [47]:
len(df["genres"].unique())

1827

-------------
<a id='Improving_Performance2'></a>
## <font color='red'>problem2:</font> Handling Duplicates in 'genres' column
See problem in page: [Problem2: Improving Performance](#Improving_Performance)

We found that there is a duplication problem in the column,

There are genres duplications, and also redundant commas

In [48]:
test = [val for val in df["genres"].unique() if val.startswith(",")]
test[2]

', 2D, 2D, Action, Action RPG, Platformer, Platformer, Role-Playing'

Exexample:

In [49]:
df["genres"][23]

'Action, Arcade, First-Person, Sci-Fi, Sci-Fi, Shooter, Shooter'

Handling Duplicates:

In [50]:
genres_per_game = []
for game_genres in df["genres"]:
    genre_list = set(game_genres.strip(", ").lower().split(", "))
    genre_list = sorted(list(genre_list))
    genres_new_string = ", ".join(genre_list)
    genres_per_game.append(genres_new_string)
    
df["genres"] = genres_per_game

In [51]:
len(df["genres"].unique())

1371

In [52]:
df["genres"][23]

'action, arcade, first-person, sci-fi, shooter'

----------------
#### Reordering the dataframe

In [53]:
columns_order = ["game_name", "platform", "genres",
                 "develeoper", "publisher", "max_players",
                 "online_game", "release_year", "release_month",
                 "rating", "meta_score", "user_score",
                 "critic_positive_normelize", "user_positive_normelize",
                ]
df = df[columns_order]
df.head()

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize
0,The Legend of Zelda: Ocarina of Time,Nintendo,"action adventure, fantasy",Nintendo,Nintendo,1,0,1998,11,1,1.000000,0.936170,1.0,0.893962
1,Tony Hawk's Pro Skater 2,PlayStation,"alternative, skateboarding, sports",Neversoft Entertainment,Activision,2,0,2000,9,3,0.988636,0.765957,1.0,0.699196
2,Grand Theft Auto IV,PlayStation,"action adventure, modern, open-world",Rockstar North,Rockstar Games,1,0,2008,4,4,0.988636,0.797872,1.0,0.714730
3,SoulCalibur,Old Platform,"3d, action, fighting",Namco,Namco,2,0,1999,9,3,0.988636,0.861702,1.0,0.825123
4,Grand Theft Auto IV,Xbox,"action adventure, modern, open-world",Rockstar North,Rockstar Games,1,0,2008,4,4,0.988636,0.819149,1.0,0.727593


----------------

## We have meny duplicates in the 'game_name' column

In [54]:
df[df["game_name"] == "Assassin's Creed Origins"]

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize
1674,Assassin's Creed Origins,PC,"action adventure, edutainment, miscellaneous, ...",Ubisoft,Ubisoft,0,0,2017,10,4,0.829545,0.702128,0.896552,0.584201
3227,Assassin's Creed Origins,PlayStation,"action adventure, edutainment, miscellaneous, ...",Ubisoft,Ubisoft,0,0,2017,10,4,0.795455,0.744681,0.841270,0.640732


------------------

## Handling duplicates in 'game_name'

In [55]:
duplicates = df[df["game_name"].duplicated(keep=False)].copy()
duplicates_names = list(duplicates["game_name"].unique())
df.reset_index(drop=True, inplace=True)

for duplicate_name in duplicates_names:
    duplicates_game = df[df["game_name"] == duplicate_name]
    
    # saving the mean valu of the duplicates
    meta_score_mean = duplicates_game["meta_score"].mean()
    user_score_mean = duplicates_game["user_score"].mean()
    critic_mean = duplicates_game["critic_positive_normelize"].mean()
    user_mean = duplicates_game["user_positive_normelize"].mean()
    
    row_index = df[df["game_name"] == duplicate_name].index[0]
    df.loc[row_index, "meta_score"] = meta_score_mean
    df.loc[row_index, "user_score"] = user_score_mean
    df.loc[row_index, "critic_positive_normelize"] = critic_mean
    df.loc[row_index, "user_positive_normelize"] = user_mean
    
    # making a platform string of all the duplicates platforms
    platforms = set()
    for val in duplicates_game["platform"]:
        platforms.add(val)
    # the list will be sorted for future labeling
    platforms = sorted(list(platforms))
    df.loc[row_index, "platform"] = ", ".join(platforms)
    
    # saving the earliest date of the duplicates
    year_month_zip = zip(duplicates_game["release_year"], duplicates_game["release_month"])
    release_year = min(duplicates_game["release_year"])
    release_month = 12
    for year, month in year_month_zip:
        if(year == release_year and month < release_month):
            release_month = month
    df.loc[row_index, "release_year"] = release_year
    df.loc[row_index, "release_month"] = release_month

df = df.drop_duplicates(subset="game_name")
df.reset_index(drop=True, inplace=True)

In [56]:
len(df["platform"].unique())

54

-------------------

## Making 'exclusive_game' column:
Content:

1 - Games released for only one platform

0 - Otherwise

In [57]:
exclusive_games_list = []
for platforms in df["platform"]:
    platforms_list = platforms.split(", ")
    if(len(platforms_list) != 1 or 'Old Platform' in platforms_list):
        exclusive_games_list.append(0)
    else:
        exclusive_games_list.append(1)
df['exclusive_game'] = exclusive_games_list

After Duplication Handling

In [58]:
df[df["game_name"] == "Assassin's Creed Origins"]

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game
1223,Assassin's Creed Origins,"PC, PlayStation","action adventure, edutainment, miscellaneous, ...",Ubisoft,Ubisoft,0,0,2017,10,4,0.8125,0.723404,0.868911,0.612466,0


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11695 entries, 0 to 11694
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  11695 non-null  object 
 1   platform                   11695 non-null  object 
 2   genres                     11695 non-null  object 
 3   develeoper                 11695 non-null  object 
 4   publisher                  11695 non-null  object 
 5   max_players                11695 non-null  int64  
 6   online_game                11695 non-null  int64  
 7   release_year               11695 non-null  int64  
 8   release_month              11695 non-null  int64  
 9   rating                     11695 non-null  int64  
 10  meta_score                 11695 non-null  float64
 11  user_score                 11695 non-null  float64
 12  critic_positive_normelize  11695 non-null  float64
 13  user_positive_normelize    11695 non-null  flo

---------------

## Labeling the 'publisher', 'develeoper', 'genres', 'main_genre' and 'platform' columns

In [60]:
df["publisher_labeled"] = LabelEncoder().fit_transform(df["publisher"])
df["develeoper_labeled"] = LabelEncoder().fit_transform(df["develeoper"])
df["genres_labeled"] = LabelEncoder().fit_transform(df["genres"])
df["platform_labeled"] = LabelEncoder().fit_transform(df["platform"])

In [61]:
df

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game,publisher_labeled,develeoper_labeled,genres_labeled,platform_labeled
0,The Legend of Zelda: Ocarina of Time,Nintendo,"action adventure, fantasy",Nintendo,Nintendo,1,0,1998,11,1,1.000000,0.936170,1.000000,0.893962,1,1070,2078,262,0
1,Tony Hawk's Pro Skater 2,"Nintendo, Old Platform, PC, PlayStation","alternative, skateboarding, sports",Neversoft Entertainment,Activision,2,0,2000,9,3,0.931818,0.765957,0.970909,0.717737,0,59,2045,805,3
2,Grand Theft Auto IV,"PC, PlayStation, Xbox","action adventure, modern, open-world",Rockstar North,Rockstar Games,1,0,2008,4,4,0.958333,0.780142,0.983333,0.681603,0,1323,2572,323,43
3,SoulCalibur,"Old Platform, Xbox","3d, action, fighting",Namco,Namco,2,0,1999,9,3,0.880682,0.829787,0.947368,0.725062,0,1042,2013,143,38
4,Super Mario Galaxy,Wii,"3d, action, platformer",Nintendo,Nintendo,0,0,2007,11,1,0.977273,0.936170,1.000000,0.911193,1,1070,2078,154,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11690,SPOGS Racing,Wii,"arcade, automobile, driving, racing",Pronto Games,D2C Games,0,0,2008,7,1,0.079545,0.361702,0.000000,0.363636,1,373,2380,834,51
11691,Double Dragon II: Wander of the Dragons,Xbox,"2d, action, beat-'em-up",Gravity Corporation,CFK,0,0,2013,4,3,0.068182,0.138298,0.000000,0.135593,1,256,1258,33,53
11692,Vroom in the Night Sky,Nintendo,"biking, individual, sports",Poisoft,Poisoft,0,0,2017,4,1,0.068182,0.319149,0.000000,0.284404,1,1221,2346,910,0
11693,Yaris,Xbox,"arcade, automobile, driving, racing",Backbone Entertainment,Backbone Entertainment,2,1,2007,10,2,0.068182,0.436170,0.000000,0.389706,1,143,277,834,53


----------------
#### Reordering the dataframe

In [62]:
columns_order = ["game_name", "platform", "genres",
                 "develeoper", "publisher", "max_players",
                 "online_game", "release_year", "release_month",
                 "rating", "meta_score", "user_score",
                 "critic_positive_normelize", "user_positive_normelize",
                 "exclusive_game", "publisher_labeled", "develeoper_labeled",
                 "genres_labeled", "platform_labeled",
                ]
df = df[columns_order]

# Final result

In [63]:
df.head()

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game,publisher_labeled,develeoper_labeled,genres_labeled,platform_labeled
0,The Legend of Zelda: Ocarina of Time,Nintendo,"action adventure, fantasy",Nintendo,Nintendo,1,0,1998,11,1,1.000000,0.936170,1.000000,0.893962,1,1070,2078,262,0
1,Tony Hawk's Pro Skater 2,"Nintendo, Old Platform, PC, PlayStation","alternative, skateboarding, sports",Neversoft Entertainment,Activision,2,0,2000,9,3,0.931818,0.765957,0.970909,0.717737,0,59,2045,805,3
2,Grand Theft Auto IV,"PC, PlayStation, Xbox","action adventure, modern, open-world",Rockstar North,Rockstar Games,1,0,2008,4,4,0.958333,0.780142,0.983333,0.681603,0,1323,2572,323,43
3,SoulCalibur,"Old Platform, Xbox","3d, action, fighting",Namco,Namco,2,0,1999,9,3,0.880682,0.829787,0.947368,0.725062,0,1042,2013,143,38
4,Super Mario Galaxy,Wii,"3d, action, platformer",Nintendo,Nintendo,0,0,2007,11,1,0.977273,0.936170,1.000000,0.911193,1,1070,2078,154,51


In [64]:
df.iloc[:, 5:]

,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game,publisher_labeled,develeoper_labeled,genres_labeled,platform_labeled
0,1,0,1998,11,1,1.000000,0.936170,1.000000,0.893962,1,1070,2078,262,0
1,2,0,2000,9,3,0.931818,0.765957,0.970909,0.717737,0,59,2045,805,3
2,1,0,2008,4,4,0.958333,0.780142,0.983333,0.681603,0,1323,2572,323,43
3,2,0,1999,9,3,0.880682,0.829787,0.947368,0.725062,0,1042,2013,143,38
4,0,0,2007,11,1,0.977273,0.936170,1.000000,0.911193,1,1070,2078,154,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11690,0,0,2008,7,1,0.079545,0.361702,0.000000,0.363636,1,373,2380,834,51
11691,0,0,2013,4,3,0.068182,0.138298,0.000000,0.135593,1,256,1258,33,53
11692,0,0,2017,4,1,0.068182,0.319149,0.000000,0.284404,1,1221,2346,910,0
11693,2,1,2007,10,2,0.068182,0.436170,0.000000,0.389706,1,143,277,834,53


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11695 entries, 0 to 11694
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   game_name                  11695 non-null  object 
 1   platform                   11695 non-null  object 
 2   genres                     11695 non-null  object 
 3   develeoper                 11695 non-null  object 
 4   publisher                  11695 non-null  object 
 5   max_players                11695 non-null  int64  
 6   online_game                11695 non-null  int64  
 7   release_year               11695 non-null  int64  
 8   release_month              11695 non-null  int64  
 9   rating                     11695 non-null  int64  
 10  meta_score                 11695 non-null  float64
 11  user_score                 11695 non-null  float64
 12  critic_positive_normelize  11695 non-null  float64
 13  user_positive_normelize    11695 non-null  flo

In [66]:
df.describe(include="all")

,game_name,platform,genres,develeoper,publisher,max_players,online_game,release_year,release_month,rating,meta_score,user_score,critic_positive_normelize,user_positive_normelize,exclusive_game,publisher_labeled,develeoper_labeled,genres_labeled,platform_labeled
count,11695,11695,11695,11695,11695,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.000000,11695.00000
unique,11695,54,1296,3575,1819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,The Legend of Zelda: Ocarina of Time,PC,"2d, action, platformer",Capcom,Nintendo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,2710,476,192,563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.459855,0.259000,2011.186575,6.972980,2.529372,0.670451,0.708804,0.481108,0.590597,0.677555,904.794699,1774.533048,666.640445,35.26832
std,NaN,NaN,NaN,NaN,NaN,6.249083,0.438104,6.401113,3.345524,1.056537,0.138328,0.140259,0.309747,0.185361,0.467432,533.328536,1027.786135,391.251545,17.31440
min,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1996.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,2006.000000,4.000000,1.000000,0.594223,0.638298,0.200000,0.473684,0.000000,456.000000,903.000000,322.000000,29.00000
50%,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,2011.000000,7.000000,3.000000,0.693182,0.739362,0.482517,0.619023,1.000000,964.000000,1759.000000,685.000000,39.00000
75%,NaN,NaN,NaN,NaN,NaN,2.000000,1.000000,2017.000000,10.000000,3.000000,0.768939,0.808511,0.750000,0.731707,1.000000,1373.000000,2685.000000,1042.000000,47.00000


## Saving data in 'all_games.csv' file

In [67]:
# df.to_csv(f'games_data/lowest_top_merge.csv')

<a id='Incomplete_data'></a>
## Saving data in 'handled_games.csv' file
### Added after <font color='red'>Problem1:</font> Incomplete data

In [68]:
# df.to_csv(f'games_data/handled_games.csv')

<a id='Improving_Performance4'></a>
## Saving data in 'handled_games2.csv' file
### Added after <font color='red'>Problem2:</font> Improving Performance

In [69]:
df.to_csv(f'games_data/handled_games2.csv')

After saving the file we moved to the next step: EDA

see next step: [4_EDA](./4_EDA.ipynb)

-------------------